In [ ]:
def sigmoid(z):
  return 1 / (1 + np.exp(-z))

In [ ]:
def softmax(z):
  exp_z = np.exp(z)
  return (exp_z / np.sum(exp_z, axis=0, keepdims=True))

In [ ]:
def relu(z):
  return (np.maximum(0, z))

In [ ]:
def relu_derivative(z):
  return np.array(z > 0)

In [ ]:
def leaky_relu(z):
  return np.maximum(0.01*z, z)

In [ ]:
def leaky_relu_derivative(z):
  return np.array(z > 0)

In [ ]:
def initial_parameters(layers):
  parameters = {}

  for i in range(1, len(layers)):
    parameters[f"w{i}"] = np.random.randn(layers[i], layers[i - 1]) / np.sqrt(layers[i - 1])
    parameters[f"b{i}"] = np.zeros((layers[i], 1))

  return parameters

In [ ]:
def forward_propagation(x, parameters, activation):
  L = len(parameters) // 2
  forward_cache = {}
  forward_cache = {"a0": x}

  for i in range(1, L):
    forward_cache[f"z{i}"] = np.dot(parameters[f"w{i}"], forward_cache[f"a{i-1}"]) + parameters[f"b{i}"]
    a = activation[i - 1](forward_cache[f"z{i}"])
    forward_cache[f"a{i}"] = a

  forward_cache[f"z{L}"] = np.dot(parameters[f"w{L}"] ,forward_cache[f"a{L - 1}"]) + parameters[f"b{L}"]

  if forward_cache[f"z{L}"].shape[0] == 1:
    forward_cache[f"a{L}"] = sigmoid(forward_cache[f"z{L}"])

  else:
    forward_cache[f"a{L}"] = softmax(forward_cache[f"z{L}"])

  return forward_cache[f"a{L}"], forward_cache

In [ ]:
def nn_cost(a,y):
  m = y.shape[1]
  if y.shape[0] == 1:
    cost = -(1 / m) * np.sum(y * np.log(a) + (1 - y) * np.log(1 - a))

  else:
    cost = -(1 / m) * np.sum(y * np.log(a))

  cost = np.squeeze(cost)
  return cost

In [ ]:
def backward_propagation(a,y, parameters, forward_cache, activation_derivative):
  gradient = {}
  m = y.shape[1]
  L = len(parameters) // 2

  gradient[f"dz{L}"] = a - y
  gradient[f"dw{L}"] = (1 / m) * np.dot(gradient[f"dz{L}"], forward_cache[f"a{L - 1}"].T)
  gradient[f"db{L}"] = (1 / m) * np.sum(gradient[f"dz{L}"], axis=1, keepdims=True)

  for i in reversed(range(1, L)):
    gradient[f"dz{i}"] = np.dot(parameters[f"w{i + 1}"].T, gradient[f"dz{i + 1}"]) * activation_derivative[i - 1](forward_cache[f"a{i}"])
    gradient[f"dw{i}"] = (1 / m) * np.dot(gradient[f"dz{i}"], forward_cache[f"a{i - 1}"].T)
    gradient[f"db{i}"] = (1 / m) * np.sum(gradient[f"dz{i}"], axis=1, keepdims=True)

  return gradient

In [ ]:
def update_parameter(parameters, gradient, alpha):
  L = len(parameters) // 2

  for i in range(1, L + 1):
    parameters[f"w{i}"] -= alpha * gradient[f"dw{i}"]
    parameters[f"b{i}"] -= alpha * gradient[f"db{i}"]

  return parameters

In [ ]:
def nn(x, y, layers, activation, activation_derivative, alpha, num_itr):
  parameters = initial_parameters(layers)

  for i in range(num_itr):
    a , forward_cache = forward_propagation(x, parameters, activation)
    cost = nn_cost(a, y)
    gradient = backward_propagation(a, y, parameters, forward_cache, activation_derivative)
    parameters = update_parameter(parameters, gradient, alpha)

    if i % 100 == 0:
      print(f"Iteration {i} \t Cost: {cost}")

  return parameters

In [ ]:
def accuracy(x,y,parameters,activaion):
  m,n = y.shape
  predics, forward_cache = forward_propagation(x, parameters, activation)
  if m == 1:
    predics = np.array(predics > 0.5 ,dtype = 'float')

  else:
    y = np.argmax(y, axis=0)
    predics = np.argmax(predics, axis=0)

  return (np.sum(y == predics)/n)

In [ ]:
df = pd.read_csv('/content/multi_classification_train.csv')
df = df.iloc[:500,1:]
m,n = df.shape
n = n-1
x = np.zeros((n,m))
y = np.zeros((1,m))
y[0] = df.iloc[:,-1]
for i in range(n):
  x[i] = df.iloc[:,i]

In [ ]:
def nn_train_data(path):
  path = path.strip("'")
  df = pd.read_csv(path)

  u,v = df.shape
  v -= 2
  a_start = int(input("give the starting index of training data:"))
  a_end = int(input("give the ending index of training data:"))
  a = a_end - a_start
  x = np.zeros((v,a))
  y = np.zeros((1,a))

  df = df.iloc[a_start:a_end,1:]
  y[0] = df[df.columns[-1]].to_numpy()
  for i in range(v):
    x[i] = df[df.columns[i]].to_numpy()

  return (x,y)

In [ ]:
x,y = nn_train_data('/content/multi_classification_train.csv')

give the starting index of training data:0
give the ending index of training data:500


In [ ]:
def multi_to_binary(y):
  a = int(y.max() + 1)
  y_new = np.zeros((a,m))
  for i in range(m):
    y_new[int(y[0,i]),i] = 1
  return y_new

In [ ]:
y = multi_to_binary(y)

In [ ]:
layers = [x.shape[0],10,20, y.shape[0]]
activation = [relu,relu,relu]
activation_derivatives = [relu_derivative,relu_derivative,relu_derivative]

parameters = nn(x,y, layers, activation, activation_derivatives, alpha=0.01, num_itr=2000)

In [ ]:
def test(x):
  a,_=forward_propagation(x,parameters,activation)
  return a

In [ ]:
y_hat = test(x)
print(y_hat)

In [ ]:
df = pd.read_csv('/content/binary_classification_train.csv')
df = df.iloc[500:700,1:]
m,n = df.shape
n = n-1
x = np.zeros((n,m))
y = np.zeros((1,m))
y[0] = df.iloc[:,-1]
for i in range(n):
  x[i] = df.iloc[:,i]

y_hat = test(x)
print(y_hat)

In [ ]:
def F1_score_binary(y_hat,y):
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  y = np.squeeze(y)
  y_hat = np.squeeze(y_hat)
  for i in range(int(len(y))):
    if y_hat[i] == y[i] == 1:
      tp += 1
    elif y_hat[i] == y[i] == 0:
      tn += 1
    elif y_hat[i] == 0 and y[i] == 1:
      fn += 1
    else:
      fp += 1

  precision = tp/(tp+fp)
  recall = tp/(tp+fn)
  F1 = 2*precision*recall/(precision+recall)
  return F1,tp,tn,fp,fn

In [ ]:
y_hat = np.squeeze(y_hat)
w = 0
for i in range(len(y_hat)):
  if y_hat[i] >= 0.5:
    y_hat[i] = 1
    w += 1
  else:
    y_hat[i] = 0
print(y_hat)
print(w)

In [ ]:
F1,F1_macro,F1_micro = F1_score_multi(y_hat,y)
print(f"F1: {F1}")
print(f"macro F1: {F1_macro}")
print(f"micro F1: {F1_micro}")